In [7]:
import polars as pl
import matplotlib.pyplot as plt
import glob
import numpy as np
import math
import os
import glob

from tqdm import tqdm


In [8]:
#pl.read_parquet("/Users/kunkerdthaisong/Downloads/dataframe.parquet")

In [9]:
df=pl.read_parquet("/Users/kunkerdthaisong/Downloads/two_person_dataframe.parquet")
df

frame,joint,skel_body,x,y,z,dis_from_sensor,file_path,filename_n_class
i64,i64,i64,f64,f64,f64,f64,str,str
1,1,0,-0.851477,0.1916033,3.384463,1.064471,"""/Users/kunkerd…","""S001C001P001R0…"
1,2,0,-0.873564,0.4063588,3.341162,1.210748,"""/Users/kunkerd…","""S001C001P001R0…"
1,3,0,-0.893265,0.6162513,3.288516,1.370976,"""/Users/kunkerd…","""S001C001P001R0…"
1,4,0,-0.887127,0.7205115,3.303393,1.448603,"""/Users/kunkerd…","""S001C001P001R0…"
1,5,0,-0.904413,0.4904004,3.197891,1.283276,"""/Users/kunkerd…","""S001C001P001R0…"
1,6,0,-0.892437,0.2878125,3.192554,1.139078,"""/Users/kunkerd…","""S001C001P001R0…"
1,7,0,-0.830653,0.092949,3.252523,0.97768,"""/Users/kunkerd…","""S001C001P001R0…"
1,8,0,-0.823994,0.056782,3.269045,0.954492,"""/Users/kunkerd…","""S001C001P001R0…"
1,9,0,-0.851896,0.546665,3.398047,1.303607,"""/Users/kunkerd…","""S001C001P001R0…"


In [11]:
df["filename_n_class"].unique(maintain_order=True)[:30]

30

In [6]:
len(df["skel_body"].unique())>1,os.path.basename(df["file_path"][0])

(True, 'S001C001P001R001A050.skeleton.npy')

In [39]:
def lin_filter(df,start:int)->int:
    for i,j in enumerate(df["filename_n_class"][start:]):
        if(df["filename_n_class"][start]!=j):
            start=start+i
            break
    return start

In [40]:
df["filename_n_class"][:3700].unique()

filename_n_class
str
"""S001C001P001R0…"


In [65]:
n=0
prv_ns=0
while(n<33):
    ns=lin_filter(df,prv_ns)
    print(type(df))
    print(len(df["filename_n_class"][prv_ns:ns].unique()))
    print(prv_ns,ns)
    n+=1
    prv_ns=ns


7300
<class 'polars.dataframe.frame.DataFrame'>
1
0 3700
<class 'polars.dataframe.frame.DataFrame'>
1
3700 7300
<class 'polars.dataframe.frame.DataFrame'>
1
7300 11000
<class 'polars.dataframe.frame.DataFrame'>
1
11000 17650
<class 'polars.dataframe.frame.DataFrame'>
1
17650 21100
<class 'polars.dataframe.frame.DataFrame'>
1
21100 25850
<class 'polars.dataframe.frame.DataFrame'>
1
25850 31600
<class 'polars.dataframe.frame.DataFrame'>
1
31600 36250
<class 'polars.dataframe.frame.DataFrame'>
1
36250 39800
<class 'polars.dataframe.frame.DataFrame'>
1
39800 44750
<class 'polars.dataframe.frame.DataFrame'>
1
44750 49400
<class 'polars.dataframe.frame.DataFrame'>
1
49400 53100
<class 'polars.dataframe.frame.DataFrame'>
1
53100 56900
<class 'polars.dataframe.frame.DataFrame'>
1
56900 60400
<class 'polars.dataframe.frame.DataFrame'>
1
60400 65050
<class 'polars.dataframe.frame.DataFrame'>
1
65050 68250
<class 'polars.dataframe.frame.DataFrame'>
1
68250 72050
<class 'polars.dataframe.frame.Dat

In [69]:
df[:131500].unique(maintain_order=True).write_parquet("/Users/kunkerdthaisong/ipu/ntu_rgb_proj/ntu_rgb/two_person_a50_a60_3.parquet")

In [71]:
df=pl.read_parquet("/Users/kunkerdthaisong/ipu/ntu_rgb_proj/ntu_rgb/two_person_a50_a60_3.parquet")
df=df.filter(pl.col("joint").is_in([]))

In [72]:
df

frame,joint,skel_body,x,y,z,dis_from_sensor,file_path,filename_n_class
i64,i64,i64,f64,f64,f64,f64,str,str
1,1,0,-0.851477,0.1916033,3.384463,1.064471,"""/Users/kunkerd…","""S001C001P001R0…"
1,2,0,-0.873564,0.4063588,3.341162,1.210748,"""/Users/kunkerd…","""S001C001P001R0…"
1,3,0,-0.893265,0.6162513,3.288516,1.370976,"""/Users/kunkerd…","""S001C001P001R0…"
1,4,0,-0.887127,0.7205115,3.303393,1.448603,"""/Users/kunkerd…","""S001C001P001R0…"
1,5,0,-0.904413,0.4904004,3.197891,1.283276,"""/Users/kunkerd…","""S001C001P001R0…"
1,6,0,-0.892437,0.2878125,3.192554,1.139078,"""/Users/kunkerd…","""S001C001P001R0…"
1,7,0,-0.830653,0.092949,3.252523,0.97768,"""/Users/kunkerd…","""S001C001P001R0…"
1,8,0,-0.823994,0.056782,3.269045,0.954492,"""/Users/kunkerd…","""S001C001P001R0…"
1,9,0,-0.851896,0.546665,3.398047,1.303607,"""/Users/kunkerd…","""S001C001P001R0…"
